# PYNQ-Z2 Bypass Effect

入力した音をそのまま出力するだけのサンプルです。ただし受信した音声を読み取って出力レジスタに書き込む処理はVivado HLSで作成したIPで自動実行されています。

## Project Repo

https://github.com/kamiyaowl/pynq_dsp_hw

## Vivado HLS IP Repo

https://github.com/kamiyaowl/pynq_dsp_hls

In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay(bitfile='/home/xilinx/dist/base_wrapper.bit', dtbo='/home/xilinx/dist/base_wrapper.dtbo', download=True)

# print(base.device.ip_dict['audio_codec_ctrl_0'])
# print(base.device.ip_dict['bypass_0'])

In [4]:
# Line入力を有効化
pAudio = base.audio
pAudio.select_line_in() # line入力を使う

# 自作HLSライブラリを設定する
bypass = base.bypass_0
offset_basePhyisAddr = bypass.register_map.basePhysAddr_V.address
offset_ctrl          = bypass.register_map.CTRL.address

# Bypassの読み書きのベースアドレスを指定
bypass.write(offset=offset_basePhyisAddr, value=pAudio.mmio.base_addr)
# HLSのモジュールを開始させる
bypass.write(offset=offset_ctrl, value=0x81) # AUTO_RESTART, AP_START
# 設定内容を表示
print(bypass.register_map)

RegisterMap {
  CTRL = Register(AP_START=1, AP_DONE=1, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  basePhysAddr_V = Register(basePhysAddr_V=1136656384)
}


In [5]:
# Auto Restartを切ってみる
bypass.write(offset=offset_ctrl, value=0x00) # AUTO_RESTART, AP_START

In [ ]:
# CPUでやる場合(止める際はInterruptしてください)

# from audio_adau1761.h
I2S_DATA_RX_L_REG = 0x00
I2S_DATA_RX_R_REG = 0x04
I2S_DATA_TX_L_REG = 0x08
I2S_DATA_TX_R_REG = 0x0C
I2S_STATUS_REG    = 0x10

pAudio = base.audio
pAudio.select_line_in() # line入力を使う

while True:
    # 音声の取得
    l_src = pAudio.mmio.read_reg(offset=I2S_DATA_RX_L_REG)
    r_src = pAudio.mmio.read_reg(offset=I2S_DATA_RX_R_REG)
    # 音声の書き込み
    pAudio.mmio.write_reg(data=l_src, offset=I2S_DATA_TX_L_REG)
    pAudio.mmio.write_reg(data=r_src, offset=I2S_DATA_TX_R_REG)